**Table of contents**<a id='toc0_'></a>    
- 1. [Analyse de Sentiment Basée sur des Tweets - Modèle Baseline](#toc1_)    
  - 1.1. [Préparation des Données](#toc1_1_)    
  - 1.2. [Vectorisation des Tweets](#toc1_2_)    
  - 1.3. [Entraînement et Évaluation du Modèle](#toc1_3_)    
  - 1.4. [Comparaison des Résultats](#toc1_4_)    
    - 1.4.1. [Comparaison des Performances entre TF-IDF et Count Vectorizer](#toc1_4_1_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Analyse de Sentiment Basée sur des Tweets - Modèle Baseline](#toc0_)

Ce notebook présente un modèle de base pour l'analyse de sentiment de tweets en utilisant la **régression logistique**. Nous explorerons deux méthodes de vectorisation du texte : **TF-IDF** et **Count Vectorizer**, afin de convertir le texte en caractéristiques numériques exploitables par le modèle. 

Les principaux objectifs sont :
- Préparer les données textuelles en utilisant des méthodes de vectorisation.
- Entraîner un modèle de régression logistique sur les tweets vectorisés.
- Suivre et évaluer les performances du modèle en utilisant **MLFlow** pour chaque méthode de vectorisation.

In [ ]:
# Bibliothèque pour la manipulation des données
import pandas as pd

# Bibliothèques pour la vectorisation et la transformation des textes
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Modèle de régression logistique pour la classification
from sklearn.linear_model import LogisticRegression

# Bibliothèques pour la division des données et les métriques d'évaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Bibliothèque pour le suivi des expérimentations
import mlflow
import mlflow.sklearn

## 1.1. <a id='toc1_1_'></a>[Préparation des Données](#toc0_)

Nous chargeons le dataset prétraité, qui contient les tweets et leur sentiment associé.

In [2]:
# Charger les données prétraitées
df = pd.read_csv('data/processed_tweets.csv')
X_text = df['text']
y = df['sentiment']

In [3]:
df.head()

,sentiment,text
0,0,awww thats bummer shoulda got david carr third...
1,0,upset cant update facebook texting might cry r...
2,0,dived many time ball managed save 50 rest go b...
3,0,whole body feel itchy like fire
4,0,behaving im mad cant see


In [4]:
# Initialiser MLFlow et Démarrer une Expérimentation
mlflow.set_experiment("Sentiment_Analysis_Baseline_Model")

<Experiment: artifact_location='file:///e:/Openclassroom/AI_Engineer/Projet_07/work_folder/mlruns/233432721080566223', creation_time=1731601110797, experiment_id='233432721080566223', last_update_time=1731601110797, lifecycle_stage='active', name='Sentiment_Analysis_Baseline_Model', tags={}>

## 1.2. <a id='toc1_2_'></a>[Vectorisation des Tweets](#toc0_)

Nous utilisons **TF-IDF** et **Count Vectorizer** pour créer des représentations numériques des tweets. TF-IDF pèse l’importance des mots en fonction de leur fréquence, tandis que Count Vectorizer compte les occurrences de chaque mot. Nous limiterons les caractéristiques à un maximum de 5000 mots pour optimiser le modèle.

In [5]:
# Créer les Features avec TF-IDF et Count Vectorizer
vectorizer_tfidf = TfidfVectorizer(max_features=5000)
vectorizer_count = CountVectorizer(max_features=5000)

# Vectoriser les textes en utilisant TF-IDF
X_tfidf = vectorizer_tfidf.fit_transform(X_text)
# Vectoriser les textes en utilisant Count Vectorizer
X_count = vectorizer_count.fit_transform(X_text)

# Diviser les données en jeu d'entraînement et de test pour chaque méthode
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
X_train_count, X_test_count, _, _ = train_test_split(X_count, y, test_size=0.2, random_state=42)

## 1.3. <a id='toc1_3_'></a>[Entraînement et Évaluation du Modèle](#toc0_)

Dans cette section, nous entraînons un modèle de **régression logistique** pour chaque méthode de vectorisation. Nous utilisons MLFlow pour suivre les expérimentations et enregistrer les métriques clés, telles que la précision, le F1-score et le ROC-AUC, afin de comparer les performances des deux méthodes de vectorisation.

In [6]:
# Entraîner le Modèle et Suivre l'Expérimentation avec MLFlow
# Utiliser MLFlow pour suivre les expérimentations avec les deux vectorisations
for vectorizer_name, X_train, X_test in [("TF-IDF", X_train_tfidf, X_test_tfidf), ("Count Vectorizer", X_train_count, X_test_count)]:
    with mlflow.start_run():
        # Entraîner le modèle de Régression Logistique
        model = LogisticRegression()
        model.fit(X_train, y_train)

        # Prédictions et Probabilités pour le ROC-AUC
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilité de la classe positive

        # Évaluation des performances
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)  # Calcul du ROC-AUC
        report = classification_report(y_test, y_pred, output_dict=True)

        # Enregistrer les paramètres et les métriques dans MLFlow
        mlflow.log_param("vectorizer", vectorizer_name)
        mlflow.log_param("model_type", "Logistic Regression")
        mlflow.log_param("max_features", 5000)

        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("roc_auc", roc_auc)  # Enregistrement du ROC-AUC
        mlflow.log_metric("precision", report["weighted avg"]["precision"])
        mlflow.log_metric("recall", report["weighted avg"]["recall"])
        mlflow.log_metric("f1-score", report["weighted avg"]["f1-score"])

        # Définir un exemple d'entrée pour MLFlow
        input_example = X_test[0].toarray()  # Convertir le premier tweet vectorisé en format array

        # Enregistrer le modèle dans MLFlow avec un exemple d'entrée
        mlflow.sklearn.log_model(model, "model", input_example=input_example)

        print(f"Results for {vectorizer_name}:")
        print("Accuracy:", accuracy)
        print("ROC-AUC:", roc_auc)
        print("Classification Report:\n", classification_report(y_test, y_pred))

Results for TF-IDF:
Accuracy: 0.774974111149465
ROC-AUC: 0.8550506385425456
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.76      0.77    158794
           1       0.77      0.79      0.78    159876

    accuracy                           0.77    318670
   macro avg       0.78      0.77      0.77    318670
weighted avg       0.78      0.77      0.77    318670



Results for Count Vectorizer:
Accuracy: 0.7747356199202937
ROC-AUC: 0.8492340943718327
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.74      0.77    158794
           1       0.76      0.81      0.78    159876

    accuracy                           0.77    318670
   macro avg       0.78      0.77      0.77    318670
weighted avg       0.78      0.77      0.77    318670



## 1.4. <a id='toc1_4_'></a>[Comparaison des Résultats](#toc0_)

Nous comparons les performances obtenues avec TF-IDF et Count Vectorizer en utilisant les métriques enregistrées dans MLFlow. Cela nous permet de déterminer quelle méthode de vectorisation est la plus adaptée pour notre analyse de sentiment.

In [7]:
# Nom de l'expérience
experiment_name = "Sentiment_Analysis_Baseline_Model"
experiment = mlflow.get_experiment_by_name(experiment_name)

# Récupérer les exécutions de l'expérience
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id], order_by=["start_time DESC"])

# Extraire les métriques pour chaque vectorisation
metrics = []
for _, run in runs.iterrows():
    metrics.append({
        "Vectorizer": run["params.vectorizer"],
        "Accuracy": run["metrics.accuracy"],
        "ROC-AUC": run["metrics.roc_auc"],
        "Precision": run["metrics.precision"],
        "Recall": run["metrics.recall"],
        "F1-Score": run["metrics.f1-score"]
    })

# Créer un DataFrame pour afficher les résultats
comparison_df = pd.DataFrame(metrics)
display(comparison_df)

,Vectorizer,Accuracy,ROC-AUC,Precision,Recall,F1-Score
0,Count Vectorizer,0.774736,0.849234,0.775704,0.774736,0.774505
1,TF-IDF,0.774974,0.855051,0.775328,0.774974,0.774882


### 1.4.1. <a id='toc1_4_1_'></a>[Comparaison des Performances entre TF-IDF et Count Vectorizer](#toc0_)

En comparant les deux méthodes de vectorisation :

- **Précision** : Les deux vectorisations montrent une précision similaire.
- **ROC-AUC** : TF-IDF montre un léger avantage en ROC-AUC, ce qui indique une meilleure capacité à distinguer les sentiments.
- **F1-Score** : Les F1-scores sont également très proches, ce qui signifie que les deux méthodes capturent bien les tweets positifs et négatifs.

Ces résultats indiquent que les deux vectorisations sont performantes, mais TF-IDF semble légèrement mieux adapté pour notre modèle de base.